# 使用PyQt升级MFC应用

## 摘要

MFC是比较旧的一套界面库。随着技术的发展，出现了新一代界面库-Qt。它具有很多优点。但因为由C++编写，需要一定技术基础才能使用。Python是易于使用的解释型语言。PyQt框架结合的Python的易用和Qt的强大，具有很好的应用前景。本文结合实际项目，从多个方面论述了如何将一个MFC应用移植到PyQt框架，并进行了总结。

## 背景

C是一个通用的过程式语言, 它具有静态类型, 支持结构化编程[ref](https://en.wikipedia.org/wiki/C_(programming_language)). 随着时间的演变, C语言成为了程序间的通用语言, 每个编程语言必须支持C才能与系统交互. C语言是如此的旧, 存在各种遗留语法和默认标准, 以至于添加新功能变得十分困难--因为很多程序已经默认它不会改变. 事实上, 只有编译器才能解析C语言程序; 并且, C语言没有固定的ABI或内存布局[ref](https://gankra.github.io/blah/c-isnt-a-language/).

C++作为C的扩展, 加入了对象和标准库[ref](https://en.wikipedia.org/wiki/C%2B%2B). MFC用C++的形式封装了Windows API, 于1992年发布, 作为Windows界面编程的事实标准.

随着时间的发展, 出现了其他界面框架, 如wxWidgets和Qt. Qt4.0在2005年发布, 同时支持Windows和Linux. 它的消息系统使它广受欢迎.

Python是一个高阶的通用编程语言, 具有动态类型和自动垃圾回收, 使用Python编写的程序具有清晰的逻辑和很好的可读性. PyQt以插件的形式, 使人们在Python中可以使用Qt库. 

## 程序分析与移植

### C++ Qt与PyQt的对比

如果使用C++开发Qt，需要先下载Qt库和头文件。一般通过Qt维护工具安装。开发环境一般为Qt Creator。

使用PyQt与平常开发Python相同。调试过程为一般Python调试过程。

因此，如果使用C++，不仅要学习一门非常困难的语言，而且还要学习一个新的编辑器。此外，C++的编译特性使它难以调试，每次修改代码后都要重新编译。

C++的部署方法一般为使用Release模式编译后再进行部署。这个模式通常会对程序进行优化，去掉一部分调试信息。对部署后的程序调试更为困难。虽然C++也支持远程调试，但要安装额外的工具，并且需要pdb信息。

因此，使用PyQt开发具有简单、快速、可靠的优点。

### 程序概况

此项目为录像应用, 使用MFC/C++编写, VS2010开发, 运行于车载Windows系统. 具有录像和记录运行信息功能. 其中，录像功能使用FFmpeg库实现, 包含自定义的调用FFmpeg API的函数；视频浏览使用摄像头厂家提供的SDK，使用C++调用；机车运行信息由连接到机车设备上的串口读取，传感器信息由连接到传感器的串口读取，使用Win32 API读取。机车两端的程序通过交换机连接, 通过socket通信。

视频浏览库需要封装以供Python使用；录像功能的依赖库过旧需要替代. socket通信使用网络服务替代. 串口由自定义封装.

### 底层库封装

串口，Win32接口，boost库。

boost-python对封装的简化。

使用boost-asio重写串口库；参考相关文档，将视频预览过程独立出来；最后使用boost-python创建python接口。

封装后的使用方法。

### 客户端相互通信

socket通信，HTTP通信。数据格式。

使用fastapi搭建小型服务器，通过访问服务器来获取数据。数据采用json格式。

python对通信的简化。

### 录像功能

通过查找FFmpeg相关文档，得知使用独立FFmpeg程序，配合相关参数，可以实现相同功能，不必调用底层库。

视频数据由FFmpeg进程保存，并自动切分。机车运行数据由串口读取，存储为二进制文件，放在每天的目录中。

对录像过程的简化。

### 界面设计

整体界面为一个多选项卡的界面。

视频选项卡的构建通过编写视频窗口类，然后加入网格布局。通过监听子窗口的双击事件，隐藏或显示其他窗口，来实现窗口的最大化和最小化。

设备界面选项卡由几个表格组成。通过监听相关消息来更新表中显示的文字。其他选项卡与此类似。

MFC选项卡与Qt选项卡的对比；MFC表格与Qt表格的对比。

MFC中的数据更新方式与Qt的对比，MFC窗口切换与Qt窗口切换的对比。

## 静态封装Python

PyOxidizer可以创建嵌入Python的应用, 它的目的是简化打包和分发问题, 使应用程序维护者关注于构建应用而不是把时间花在构建系统和打包工具上.

PyOxidizer甚至可以创建单文件应用-将Python和它的所有依赖静态链接, 并且将pyc等资源文件嵌入到可执行程序中.

## 文档与测试

C++中的文档框架，python中的文档框架，自动生成，比较。

Sphinx框架可以生成优雅的文档，原来用于为Python自身生成文档。它具有如下功能：
1. 支持HTML、LaTeX、ePub等输出格式
1. 支持通过语法标记来对函数、类、引用、术语自动生成交叉链接
1. 支持树形层级结构
1. 自动生成目录
1. 通过Pygments对代码进行语法高亮
1. 插件支持：对代码块进行自动测试，从pyhon代码中添加文档等
1. 可以从社区安装更多插件


根据Sphinx文档，使用Sphinx的方法为：
1. 安装：`pip install -U sphinx`
1. 生成文档基本结构: `sphinx-quickstart`
1. 调整conf.py的配置
1. 编写文档, 并在index.rst中引用.

 通过使用插件sphinx.ext.autodoc, 可以在文件中引用Python模块中的文档注释. 使用`sphinx-quickstart`中生成的`make.bat`和Makefile可以构建文档. 如`make html`可以生成HTML网页.

集成测试理论，冒烟测试（smoke test），模拟测试（mock test）。

使用vlc与python绑定python-vlc，播放本地视频，模拟摄像头SDK。

通过使用diskpart创建虚拟磁盘，通过磁盘的挂载与卸载来模拟硬盘和U盘。

## 持续集成

持续集成理论，各种集成工具与服务。

GitHub Actions，构建程序，生成文档，测试，打包。

## 总结

本论文针对原有MFC程序的功能，探讨了将各个部分的迁移方法，得出了迁移后的程序原型，并使用现代软件工程理论保证了软件的可靠性。

## Q & A

问题1：课题的研究背景是什么？ 

答：Qt迅速发展，新一代界面框架的产生 

问题2：课题的研究意义是什么？ 

答：提供将MFC应用迁移到Qt的思路和步骤 

问题3：请概述你的研究内容？ 

答：Python C++插件、Qt界面、Qt消息、集成测试。 

问题4：你认为你在研究过程中关键点是什么？ 

答：Qt消息机制应用、C++封装。 

问题5：你是怎么安排你的研究进度的？ 

答： 

Xx-xx 理论研究 

xx-xx 界面设计/各个部分的迁移 

xx-xx 硬件模拟/集成测试 

xx-xx  PPT/文档撰写

## 开题报告

### 目的 & 意义

意义：提升软件可靠性，缩短开发过程，加强软件扩展性

### 思路 & 方法

通过文档和网络搜索。

### 拟解决的关键问题

C++代码封装 

自适应UI 

硬件模拟、集成测试 

### 主要研究内容（提纲）& 任务要求

MFC开发过程

PyQt开发过程

界面设计

通信

C++插件

硬件模拟 & 集成测试

### 文献综述（国内外研究情况及其发展）& 参考文献

Qt是一个组件套装, 用于创建图形界面和跨平台应用, 可以在Linux, Windows, macOS, Android或嵌入式系统中运行. Qt当前由QT公司开发, 有商业和开源许可证。

MFC是一个C++对象库，用于在Windows上开发桌面应用。MFC在1992年由微软提出，并迅速获得了广泛的应用。虽然之后出现了很多框架，MFC仍然有广泛的使用。

串口是指一种串行通信接口，每次顺序传输1比特的信息。与它相对的是可以同时传输多个比特的并行接口。在计算机的大部分历史上，信息由串口传输到设备上，如调制解调器，终端，外围设备，以及计算机与计算机间的直接通信。

由于网口、USB接口也通过串流发送数据，串口通常指与RS-232、RS-485、RS-222或类似标准兼容的设备。

现代计算机已经将串口替换为了速度更高的设备，主要是USB。然而，串口仍然在简单、低速的接口中使用，如工业自动化系统，科学仪器，POS机等。

Python是一个高阶的通用编程语言，它的设计着重于以缩进来增加代码可读性。它的语法和面向对象的设计旨在帮助程序员编写清晰，逻辑性强的程序，从小型到大型项目。

Python具有动态类型和垃圾回收机制。它支持多种编程范式，包括过程式、面向对象式和函数式。由于python全面的标准库，它又被称为”功能齐备“的语言。

由于C/C++的悠久历史，很多功能已经由C/C++实现。Python绑定可以实现从Python中调用C/C++库。

Python对C++的调用通常需要三个步骤：将Python参数转换为C类型、加载库并调用函数、将结构转换为Python类型。boost-python库提供了一系列宏和函数，可以将数据在C/C++和Python间无缝转换，大大方便了开发过程。

软件测试是用于检测软件可靠性的实验。它也可以提供一个主动和独立的视角，用于商业来理解和接受软件实现的风险。测试方法通常包括：
1. 